In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.datasets import mnist
from tqdm import tqdm
import tensorflow as tf

In [13]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train =  (x_train.astype(np.float32) - 127.5)/127.5
x_train = x_train.reshape(60000,784)


In [16]:
x_train.shape

(60000, 784)

In [17]:
def adam_optimizer():
    return tf.keras.optimizers.Adam(lr=0.0002,beta_1=0.5)

In [18]:
def generator():
    gen = Sequential()
    gen.add(Dense(256,input_dim=100))
    gen.add(LeakyReLU(0.2))
    
    gen.add(Dense(512))
    gen.add(LeakyReLU(0.2))
    
    gen.add(Dense(1024))
    gen.add(LeakyReLU(0.2))
    
    gen.add(Dense(784,activation='tanh'))
    
    gen.compile(loss='binary_crossentropy',optimizer=adam_optimizer())
    
    return gen

In [19]:
g = generator()
g.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               8

In [20]:
def discriminator():
    disc = Sequential()
    disc.add(Dense(1024,input_dim=784))
    disc.add(LeakyReLU(0.2))
    disc.add(Dropout(0.3))
    
    disc.add(Dense(512))
    disc.add(LeakyReLU(0.2))
    disc.add(Dropout(0.3))
    
    disc.add(Dense(256,input_dim=784))
    disc.add(LeakyReLU(0.2))
    
    disc.add(Dense(1,activation='sigmoid'))
    
    disc.compile(loss='binary_crossentropy',optimizer=adam_optimizer())
    return disc

In [21]:
d = discriminator()
d.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

In [22]:
def gan(disc,gen):
    disc.trainable=False
    ip = Input(shape=(100,))
    x = gen(ip)
    op = disc(x)
    gan = Model(inputs= ip , outputs=op)
    gan.compile(loss='binary_crossentropy',optimizer='adam')
    return gan

In [23]:
gan = gan(d,g)
gan.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 784)               1486352   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________
